# Midterm
## Tasks
### 1. Make the truth tables of following statements
### with the help of PyDatalog:
* (A∨¬A)↔B
* A∨B
* (A→B)∧(B→A)
* (A∨(B∧¬A))∨(A∨¬(B∨A))

### 2. Calculate the measure of ρ(0.6/b1+0.4/b2+0.7/b3) of following relationship, 
### norm is operator of multiplication and conorm is max
### ![ro_full.png](attachment:ro_full.png)

### 3 Make the PyDatalog statements of the following anecdot story of Mark Twain:
“After long years as a bachelor I was tired of being alone and married a
widow with a grown daughter. My father fell in love with the daughter
and took her as his wife. This made me my own son-in-law and my
stepdaughter became my mother. After a year my wife gave birth to a
son. Now, my son was my father’s brother-in-law and at the same time
my uncle, since he was my stepmother’s brother. But my father’s wife also
gave birth to a son. So this was my brother and also my grandson, since
he was the son of my daughter. This meant I’d married my grandmother,
since she was the mother of my mother. As my wife’s husband, I was
also her grandson. And since the husband of a grandmother is always a
grandfather, I am my own grandfather.”

### 4 Using the scikit-fuzzy solve the following control system:
The car should move at a speed of at least 60 kmh along the street away from the traffic lights. The car should stop at the intersection on a red traffic light. If the traffic light flashes amber, then the car must cross the intersection at a speed below 40 kmh. If the rules are violated, the car gets into an accident.
The traffic light mode: red, red and yellow, green, green flashing, yellow, red, etc.
Simplified mode of traffic lights: red, yellow, green, yellow, red, etc.
The traffic light may not work (off), the traffic light may flash yellow, the traffic light may be broken (all three signals are switched on simultaneously).
The signal from the system of technical vision is not always 100% reliable.
The vision system processes video images received from a television camera. The image of the traffic light is recognized during image processing. At the output of the system, a signal is generated from the distance to the traffic light and the color signal of the traffic light. The distance signal to the traffic light is measured in meters. If the traffic light is not found or the distance to it is over 140 m, then the value of this signal is 140 m.
Antecednets (Inputs) variables:
* distance to the traffic light,
* the color of the traffic light,
* speed.

The Consequents (Outputs) variable is 
* the speed change.



In [3]:
import pandas as pd
from itertools import product

def f(a, b):
    return ((a or not a) or b) 
def truth_table(f):
    values = [list(x) + [f(*x)] for x in product([False,True], repeat=f.func_code.co_argcount)]
    return pd.DataFrame(values,columns=(list(f.func_code.co_varnames) + [f.func_name]))

truth_table(f)

AttributeError: 'function' object has no attribute 'func_code'

In [20]:
#task1
from pyDatalog import pyDatalog as pyd

values = [0, 1]
#(A∨¬A)↔B defining
def f(a, b):
    return( (not(a or not(a)) or b) and (not(b) or (a or not(a))) )
#A∨B defining
def g(a, b):
    return( a or b )
#(A→B)∧(B→A) defining
def h(a, b):
    return( (not(a) or b) and (not(b) or a) )
#(A∨(B∧¬A))∨(A∨¬(B∨A)) defining
def j(a, b):
    return( (a or (b and not(a))) or (a or not(b or a)) )
    
pyd.create_terms('X, Y, f, g, h, j, Func')
#print tables
print('Answers for (A∨¬A)↔B')
print( (X.in_(values)) & (Y.in_(values) & (Func==f(X,Y))))
print()

print('Answers for A∨B')
print( (X.in_(values)) & (Y.in_(values) & (Func==g(X,Y))))
print()

print('Answers for (A→B)∧(B→A)')
print( (X.in_(values)) & (Y.in_(values) & (Func==h(X,Y))))
print()

print('Answers for (A∨(B∧¬A))∨(A∨¬(B∨A))')
print( (X.in_(values)) & (Y.in_(values) & (Func==j(X,Y))))
print()

Answers for (A∨¬A)↔B
X | Y | Func
--|---|-----
1 | 1 | 1   
0 | 0 | 0   
1 | 0 | 0   
0 | 1 | 1   

Answers for A∨B
X | Y | Func
--|---|-----
1 | 1 | 1   
0 | 0 | 0   
0 | 1 | 1   
1 | 0 | 1   

Answers for (A→B)∧(B→A)
X | Y | Func
--|---|-----
1 | 1 | 1   
0 | 0 | 1   
1 | 0 | 0   
0 | 1 | 0   

Answers for (A∨(B∧¬A))∨(A∨¬(B∨A))
X | Y | Func
--|---|-----
1 | 0 | 1   
0 | 0 | 1   
0 | 1 | 1   
1 | 1 | 1   



In [28]:
#task2
from pyDatalog import pyDatalog as pyd

#filled table
p = [[0.5,0.2,0.4], [0.5,0.7,0.1],[0.2,0.5,0.5]] 
ans = 0 
#fill the matrix with max values
def f(b1 ,b2 ,b3): 
    return max(0.6*b1, 0.4*b2, 0.7*b3) 
#find the max value
for i in p: 
    ans = max(ans, f(i[0], i[1], i[2])) 

print("max is:", ans)

max is: 0.35


In [ ]:
#task3
from pyDatalog import pyDatalog
import logging
from pyDatalog import pyEngine
pyEngine.Logging = True
logging.basicConfig(level=logging.INFO)
pyDatalog.create_terms('link, can_reach,X,Y,Z')

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String, ForeignKey

# define a base class with SQLAlchemy and pyDatalog capabilities
Base = declarative_base(cls=pyDatalog.Mixin, metaclass=pyDatalog.sqlMetaMixin)
# create database in memory
engine = create_engine('sqlite:///:memory:', echo=False)

# open a session on a database, then associate it to the Base class
Session = sessionmaker(bind=engine)
session = Session()
Base.session = session

class Employee(Base): # Employee inherits from the Base class
    __tablename__ = 'employee' # data are stored in the Employee table

    name = Column(String, primary_key=True)
#     manager_name = Column(String, ForeignKey('employee.name'))
#     salary = Column(Integer)
    def __init__(self, name): 
        # call the initialization method of the Mixin class
        super(Employee, self).__init__()
        self.name = name
#         self.manager = manager     # direct manager of the employee, or None
#         self.salary = salary       # monthly salary of the employee
    
    def __repr__(self): # specifies how to display an Employee
        return self.name
    
# now create the table
Base.metadata.create_all(engine)
pyDatalog.create_terms('married, X, c, p')

married(X,Y) <= married(Y,X)
married(X,Y) <= p(X,Z) & p(Y, Z) & (X!=Y) #& (!married())

MT = Employee('MT')
MTFather = Employee('MTFather')
Widow = Employee('Widow')
WidowsDaughter = Employee('WidowsDaughter')
MTSon = Employee('MTSon')
MTfathersSecSon = Employee('MTfathersSecSon')